<a href="https://colab.research.google.com/github/ShovalBenjer/JSQ-SLQ/blob/main/JSQ_SLQ_Expierement_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# =============================================================================
# CELL 1: Imports, Global Parameters, and Visual Style Configuration
# =============================================================================
!pip install -q pandas numpy scipy plotly ipywidgets

import heapq
import random
from collections import deque
import numpy as np
import pandas as pd
from scipy.stats import t, ttest_ind

# --- Visualization Imports ---
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# --- Configure Professional Visual Style ---
PROFESSIONAL_PALETTE = {
    'primary': '#003f5c',      # Dark Slate Blue
    'secondary': '#ffa600',    # Muted Orange/Gold
    'accent': '#58508d',       # Muted Purple
    'neutral': '#bc5090',      # Muted Pink/Magenta
    'error_fill_primary': 'rgba(0, 63, 92, 0.2)',
    'error_fill_secondary': 'rgba(255, 166, 0, 0.2)',
}
pio.templates.default = "simple_white"

# --- Methodological Parameters ---
# Use these for the final, high-quality run (will take several hours)
STD_REPS = 50
STD_TIME = 100000

# Use these for quick checks and debugging
# LEAN_REPS = 10
# LEAN_TIME = 8000

# --- Simulation Event Constants ---
ARRIVAL, DEPARTURE, ATTEMPT_SERVICE = 'arrival', 'departure', 'attempt_service'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.2 MB/s eta 0:00:00


In [1]:
# =============================================================================
# CELL 2: Core Simulation Engine (Unified for Metrics and Animation Logging)
# =============================================================================
def run_manual_simulation(params):
    """
    Simulates a single-server, two-queue system with a k-threshold policy.

    This function is the definitive, unified simulation engine for the project.
    It can operate in two modes based on the 'log_data' parameter:
    1. Standard Mode (default): Efficiently calculates final performance metrics.
    2. Logging Mode: Additionally records the system state at regular intervals,
       which is essential for creating animations.

    The engine implements a Join-the-Shortest-Queue (JSQ) arrival policy and a
    Serve-the-K-Longer-Queue (SKLQ) service policy. It is robust against edge
    cases and returns a comprehensive set of validated performance indicators.

    Args:
        params (dict): A dictionary containing simulation parameters.

    Returns:
        tuple or dict:
            - If 'log_data' is True, returns a tuple: (metrics_dict, log_list).
            - If 'log_data' is False (default), returns only the metrics_dict.
    """
    # --- Parameter Unpacking with Defaults ---
    log_data_flag = params.get('log_data', False)
    log_interval = params.get('log_interval', 1.0)

    # --- State Variables ---
    sim_random = random.Random(params['seed'])
    now, event_calendar, queues = 0.0, [], [deque(), deque()]
    server_location, server_status, current_customer = 0, 'IDLE', None

    # --- Metric Accumulators ---
    sojourn_times = [[], []]
    switch_count = 0
    busy_time = 0.0
    bypass_count = 0

    # --- Logging Variables (only used if flag is True) ---
    system_log = []
    next_log_time = 0.0

    # --- HELPER FUNCTIONS ---
    def schedule_event(delay, event_type, data=None):
        heapq.heappush(event_calendar, (now + delay, event_type, data))

    def process_arrival(arrival_time):
        nonlocal server_location, server_status, current_customer, busy_time
        len_q1, len_q2 = len(queues[0]), len(queues[1])
        chosen_q_idx = 0 if len_q1 < len_q2 else 1 if len_q2 < len_q1 else (0 if sim_random.random() < params['p1'] else 1)
        queues[chosen_q_idx].append(arrival_time)
        schedule_event(sim_random.expovariate(params['lambda']), ARRIVAL)

        if server_status == 'BUSY' and params.get('policy') == 'preemptive' and params['k'] == 1:
            len_q1_after, len_q2_after = len(queues[0]), len(queues[1])
            preempt_needed = (server_location == 0 and len_q2_after - len_q1_after >= 1) or \
                             (server_location == 1 and len_q1_after - len_q2_after >= 1)
            if preempt_needed:
                for i, event in enumerate(event_calendar):
                    if event[1] == DEPARTURE:
                        event_calendar.pop(i); heapq.heapify(event_calendar); break
                busy_time += now - current_customer['service_start_time']
                queues[server_location].appendleft(current_customer['arrival_time'])
                server_status, current_customer = 'IDLE', None

        if server_status == 'IDLE':
            start_service_if_possible()

    def start_service_if_possible():
        nonlocal server_location, server_status, switch_count, current_customer, bypass_count
        if server_status != 'IDLE': return

        len_i, len_j = len(queues[server_location]), len(queues[1 - server_location])

        if (len_j - len_i >= params['k']):
            server_location = 1 - server_location; switch_count += 1
            schedule_event(params['switch_time'], ATTEMPT_SERVICE); return

        if len_i > 0:
            if len_j > len_i and (len_j - len_i < params['k']):
                 bypass_count += 1
            arrival_time = queues[server_location].popleft()
            service_rate = params['mu1'] if server_location == 0 else params['mu2']
            server_status, current_customer = 'BUSY', {'arrival_time': arrival_time, 'service_start_time': now}
            schedule_event(sim_random.expovariate(service_rate), DEPARTURE, {'q_idx': server_location}); return

        if len_j > 0:
            server_location = 1 - server_location; switch_count += 1
            schedule_event(params['switch_time'], ATTEMPT_SERVICE); return

    def process_departure(data):
        nonlocal server_status, busy_time, current_customer
        sojourn_times[data['q_idx']].append(now - current_customer['arrival_time'])
        busy_time += now - current_customer['service_start_time']
        server_status, current_customer = 'IDLE', None
        start_service_if_possible()

    # --- MAIN SIMULATION LOOP ---
    schedule_event(0, ARRIVAL)
    while event_calendar and now < params['sim_time']:
        if log_data_flag:
            next_event_time = event_calendar[0][0]
            while next_log_time < next_event_time and next_log_time < params['sim_time']:
                system_log.append({'time': round(next_log_time, 2), 'q1_len': len(queues[0]), 'q2_len': len(queues[1])})
                next_log_time += log_interval

        now, event_type, data = heapq.heappop(event_calendar)
        if now >= params['sim_time']: break

        if event_type == ARRIVAL: process_arrival(now)
        elif event_type == DEPARTURE: process_departure(data)
        elif event_type == ATTEMPT_SERVICE: start_service_if_possible()

    # --- FINAL METRIC CALCULATION ---
    sim_duration = params['sim_time']
    w1 = np.mean(sojourn_times[0]) if sojourn_times[0] else 0
    w2 = np.mean(sojourn_times[1]) if sojourn_times[1] else 0
    w_total = np.mean(sojourn_times[0] + sojourn_times[1]) if (sojourn_times[0] or sojourn_times[1]) else 0

    eff_lambda1, eff_lambda2, utilization = 0, 0, 0
    if sim_duration > 0:
        eff_lambda1 = len(sojourn_times[0]) / sim_duration
        eff_lambda2 = len(sojourn_times[1]) / sim_duration
        utilization = busy_time / sim_duration

    l1 = eff_lambda1 * w1; l2 = eff_lambda2 * w2

    rho1_eff = busy_time / sim_duration if server_location == 0 and busy_time > 0 else 0
    rho2_eff = busy_time / sim_duration if server_location == 1 and busy_time > 0 else 0
    lq1 = max(0, l1 - rho1_eff); lq2 = max(0, l2 - rho2_eff)

    gini = abs(w1 - w2) / (w1 + w2) if (w1 + w2) > 0 else 0

    metrics = {
        "W1": w1, "W2": w2, "W_total": w_total, "L1": l1, "L2": l2,
        "Lq1": lq1, "Lq2": lq2, "Gini": gini, "Switches": switch_count,
        "Utilization": utilization, "Bypasses": bypass_count
    }

    if log_data_flag:
        return metrics, system_log
    else:
        return metrics

In [ ]:
# =============================================================================
# CELL 3: Helper and Statistical Functions
# =============================================================================

def run_mg1_simulation(params):
    """
    Runs a basic M/G/1 queue simulation for benchmarking.
    """
    sim_random = random.Random(params['seed'])
    queue = deque(); sojourn_times = []; now = 0.0; server_status = 'IDLE'; event_calendar = []
    def schedule(d, et, dt=None): heapq.heappush(event_calendar, (now + d, et, dt))
    def p_arr(at): queue.append(at); schedule(sim_random.expovariate(params['lambda']), ARRIVAL); s_ser()
    def s_ser():
        nonlocal server_status
        if queue and server_status == 'IDLE':
            server_status = 'BUSY'; arrival_time = queue.popleft()
            st = sim_random.expovariate(params['mu1']) if sim_random.random() < params['p1'] else sim_random.expovariate(params['mu2'])
            schedule(st, DEPARTURE, {'arrival': arrival_time})
    def p_dep(dt): sojourn_times.append(now - dt['arrival']); nonlocal server_status; server_status = 'IDLE'; s_ser()
    schedule(0, ARRIVAL)
    while event_calendar and now < params['sim_time']:
        now, event_type, data = heapq.heappop(event_calendar)
        if now >= params['sim_time']: break
        if event_type == ARRIVAL: p_arr(now)
        elif event_type == DEPARTURE: p_dep(data)
    w_total = np.mean(sojourn_times) if sojourn_times else 0
    return {"L_total": params['lambda'] * w_total}

def run_replications_raw(sim_function, params, num_replications):
    """
    Runs a given simulation function multiple times with different seeds.
    Handles both standard and logging-enabled simulation runs.
    """
    results = []
    for i in range(num_replications):
        sim_output = sim_function({**params, 'seed': i})
        if isinstance(sim_output, tuple):
            results.append(sim_output[0]) # We only need the metrics dict for stats
        else:
            results.append(sim_output)
    return results

def calculate_stats(raw_results, metric_key):
    """
    Calculates the mean and 95% confidence interval half-width for a metric.
    """
    values = [res[metric_key] for res in raw_results]
    n = len(values)
    if n < 2: return {'mean': np.mean(values) if n > 0 else 0, 'ci_half_width': 0}
    mean = np.mean(values)
    std_dev = np.std(values, ddof=1)
    ci_half_width = t.ppf(0.975, n - 1) * std_dev / np.sqrt(n)
    return {'mean': mean, 'ci_half_width': ci_half_width}

In [ ]:
# =============================================================================
# CELL 4: Block 1 - Validation
# =============================================================================
def run_block_1_validation():
    """
    Validates the simulation model against data from a reference paper.
    """
    print("\n--- Running Block 1: Validation ---")
    paper_data = pd.DataFrame({
        'mu1': [3.3, 3.5, 4.0, 4.5, 5.0, 6.0, 8.0],
        'E[W1]': [32.99, 6.64, 2.25, 1.37, 1, 0.66, 0.41],
        'E[W2]': [28.72, 5.89, 2.08, 1.32, 1, 0.71, 0.49]
    })
    sim_results = []
    base_params = {'lambda': 4.0, 'mu2': 5.0, 'p1': 0.5, 'k': 1, 'switch_time': 0, 'policy': 'preemptive', 'sim_time': STD_TIME}

    for mu1 in paper_data['mu1']:
        raw = run_replications_raw(run_manual_simulation, {**base_params, 'mu1': mu1}, STD_REPS)
        sim_results.append({'mu1': mu1, 'W1_mean': np.mean([r['W1'] for r in raw]), 'W2_mean': np.mean([r['W2'] for r in raw])})
    sim_df = pd.DataFrame(sim_results)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=paper_data['mu1'], y=paper_data['E[W1]'], mode='lines', name='E[W1] (Paper)', line=dict(color=PROFESSIONAL_PALETTE['primary'], width=2)))
    fig.add_trace(go.Scatter(x=paper_data['mu1'], y=paper_data['E[W2]'], mode='lines', name='E[W2] (Paper)', line=dict(color=PROFESSIONAL_PALETTE['secondary'], width=2)))
    fig.add_trace(go.Scatter(x=sim_df['mu1'], y=sim_df['W1_mean'], mode='markers', name='E[W1] (Sim)',
                            marker=dict(color=PROFESSIONAL_PALETTE['primary'], symbol='circle-open', size=10, line=dict(width=2))))
    fig.add_trace(go.Scatter(x=sim_df['mu1'], y=sim_df['W2_mean'], mode='markers', name='E[W2] (Sim)',
                            marker=dict(color=PROFESSIONAL_PALETTE['secondary'], symbol='square-open', size=10, line=dict(width=2))))

    fig.update_layout(title_text="Block 1: Preemptive Model Validation vs. Reference Data", xaxis_title="Service Rate of Server 1 (μ₁)",
                      yaxis_title="Mean Sojourn Time (E[W])", legend_title_text='Metric')
    fig.show()

In [ ]:
# =============================================================================
# CELL 5: Block 2 - K-Threshold Analysis
# =============================================================================
def run_block_2_explore_k():
    """
    Analyzes the impact of the K-threshold on system performance and fairness.
    """
    print("\n--- Running Block 2: Impact of K-Threshold ---")
    k_range = range(1, 12)
    base_params = {'lambda': 4.0, 'mu1': 4.5, 'mu2': 5.0, 'p1': 0.5, 'switch_time': 0, 'policy': 'non_preemptive', 'sim_time': STD_TIME}
    all_raw_results = [{'K': k, **res} for k in k_range for res in run_replications_raw(run_manual_simulation, {**base_params, 'k': k}, STD_REPS)]
    df = pd.DataFrame(all_raw_results)

    agg_df = df.groupby('K').agg({m: ['mean', 'std'] for m in ['W1', 'W2', 'Lq1', 'Lq2', 'Gini', 'Bypasses']}).reset_index()
    agg_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in agg_df.columns.values]

    fig = make_subplots(rows=2, cols=2, subplot_titles=("Mean Sojourn Time (E[W])", "Mean Queue Length (E[Lq])", "Fairness (Gini Index)", "K-Induced Bypasses"))

    def plot_with_error(ax_fig, row, col, x, y_m, y_s, name, color):
        ax_fig.add_trace(go.Scatter(x=x, y=y_m, mode='lines+markers', name=name, line=dict(color=color)), row=row, col=col)
        ax_fig.add_trace(go.Scatter(x=np.concatenate([x, x[::-1]]), y=np.concatenate([y_m - y_s, (y_m + y_s)[::-1]]),
                                  fill='toself', fillcolor=color, opacity=0.2, line=dict(width=0), showlegend=False), row=row, col=col)

    plot_with_error(fig, 1, 1, agg_df['K'], agg_df['W1_mean'], agg_df['W1_std'], 'E[W1]', PROFESSIONAL_PALETTE['primary'])
    plot_with_error(fig, 1, 1, agg_df['K'], agg_df['W2_mean'], agg_df['W2_std'], 'E[W2]', PROFESSIONAL_PALETTE['secondary'])
    plot_with_error(fig, 1, 2, agg_df['K'], agg_df['Lq1_mean'], agg_df['Lq1_std'], 'E[Lq1]', PROFESSIONAL_PALETTE['primary'])
    plot_with_error(fig, 1, 2, agg_df['K'], agg_df['Lq2_mean'], agg_df['Lq2_std'], 'E[Lq2]', PROFESSIONAL_PALETTE['secondary'])

    fig.add_trace(go.Scatter(x=agg_df['K'], y=agg_df['Gini_mean'], mode='lines+markers', name='Gini Index', line=dict(color=PROFESSIONAL_PALETTE['accent'])), row=2, col=1)
    fig.add_trace(go.Bar(x=agg_df['K'], y=agg_df['Bypasses_mean'], name='Bypasses', marker_color=PROFESSIONAL_PALETTE['neutral']), row=2, col=2)

    fig.update_layout(height=800, title_text=f"SKLQ Policy Performance vs. K-Threshold ({STD_REPS} Replications)", legend_title_text="Metrics")
    fig.update_xaxes(title_text="K Threshold")
    fig.show()

In [ ]:
# =============================================================================
# CELL 6: Block 3 - Policy Comparison (Rigorous)
# =============================================================================
def run_block_5_policy_comparison():
    """
    Rigorously compares preemptive and non-preemptive policies with 95% CIs.
    """
    print("\n--- Running Block 3 (Rigorous): Preemptive vs. Non-Preemptive ---")
    st_range = [0, 0.05, 0.1, 0.15, 0.2, 0.25]
    results_pre, results_non = [], []
    base_params = {'lambda': 4.0, 'mu1': 4.0, 'mu2': 5.0, 'p1': 0.5, 'k': 1, 'sim_time': STD_TIME}

    print("Running t-tests for each switch time point...")
    for st in st_range:
        raw_pre = run_replications_raw(run_manual_simulation, {**base_params, 'policy': 'preemptive', 'switch_time': st}, STD_REPS)
        results_pre.append({'st': st, **calculate_stats(raw_pre, 'W_total')})
        raw_non = run_replications_raw(run_manual_simulation, {**base_params, 'policy': 'non_preemptive', 'switch_time': st}, STD_REPS)
        results_non.append({'st': st, **calculate_stats(raw_non, 'W_total')})

        t_stat, p_value = ttest_ind([r['W_total'] for r in raw_pre], [r['W_total'] for r in raw_non], equal_var=False)
        print(f"  Switch Time {st:.2f}: Preemptive={results_pre[-1]['mean']:.3f}, Non-Preemptive={results_non[-1]['mean']:.3f}. "
              f"Significant Difference? {'Yes' if p_value < 0.05 else 'No'} (p={p_value:.4f})")
    df_pre, df_non = pd.DataFrame(results_pre), pd.DataFrame(results_non)

    fig = go.Figure()
    def add_ci_trace(fig, df, name, color, fill, symbol, dash):
        fig.add_trace(go.Scatter(x=df['st'], y=df['mean'] + df['ci_half_width'], mode='lines', line=dict(width=0), showlegend=False))
        fig.add_trace(go.Scatter(x=df['st'], y=df['mean'], mode='lines+markers', name=name, line=dict(color=color, dash=dash),
                                  marker_symbol=symbol, fill='tonexty', fillcolor=fill))
        fig.add_trace(go.Scatter(x=df['st'], y=df['mean'] - df['ci_half_width'], mode='lines', line=dict(width=0), fill='tonexty',
                                  fillcolor=fill, showlegend=False))

    add_ci_trace(fig, df_non, 'Non-Preemptive', PROFESSIONAL_PALETTE['secondary'], PROFESSIONAL_PALETTE['error_fill_secondary'], 'square', 'dash')
    add_ci_trace(fig, df_pre, 'Preemptive', PROFESSIONAL_PALETTE['primary'], PROFESSIONAL_PALETTE['error_fill_primary'], 'circle', 'solid')

    fig.update_layout(title_text='Comparison of Policies with 95% Confidence Intervals', xaxis_title='Switch-over Time',
                      yaxis_title='Total Mean Sojourn Time (E[W_total])')
    fig.show()

In [ ]:
# =============================================================================
# CELL 7: Block 4 - Pareto Frontier Analysis
# =============================================================================
def run_pareto_frontier_analysis():
    """
    Analyzes and visualizes the trade-off between system efficiency and fairness.
    """
    print("\n--- Running Block 4: Pareto Frontier Analysis ---")
    k_range = range(1, 12)
    base_params = {'lambda': 4.0, 'mu1': 4.5, 'mu2': 5.0, 'p1': 0.5, 'switch_time': 0, 'policy': 'non_preemptive', 'sim_time': STD_TIME}
    results = []

    for k in k_range:
        raw = run_replications_raw(run_manual_simulation, {**base_params, 'k': k}, STD_REPS)
        mean_w_total = np.mean([r['W_total'] for r in raw])
        mean_gini = np.mean([r['Gini'] for r in raw])
        efficiency = 1 / mean_w_total if mean_w_total > 0 else 0
        fairness = 1 - mean_gini
        results.append({'k': k, 'efficiency': efficiency, 'fairness': fairness})

    pareto_df = pd.DataFrame(results)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pareto_df['fairness'], y=pareto_df['efficiency'], mode='lines+markers',
                             marker=dict(size=10, color=PROFESSIONAL_PALETTE['primary']),
                             line=dict(width=2, color=PROFESSIONAL_PALETTE['primary'])))

    for i, row in pareto_df.iterrows():
        fig.add_annotation(x=row['fairness'], y=row['efficiency'], text=f"K={row['k']}",
                           showarrow=False, yshift=15)

    fig.update_layout(title_text="Pareto Frontier: The Efficiency vs. Fairness Trade-off",
                      xaxis_title="Fairness (1 - Gini Index) → Higher is Better",
                      yaxis_title="System Efficiency (1 / E[W_total]) → Higher is Better")
    fig.show()

In [ ]:
# =============================================================================
# CELL 8: Block 5 - Sensitivity Analyses (Corrected)
# =============================================================================
def run_sensitivity_traffic_intensity():
    """
    Analyzes how policy performance changes with varying traffic intensity (rho).
    """
    print("\n--- Running Sensitivity Analysis: Traffic Intensity (ρ) ---")
    st_range = [0, 0.05, 0.1, 0.15, 0.2, 0.25]
    traffic_regimes = {
        'Light (ρ≈0.55)': {'lambda': 2.5},
        'Medium (ρ≈0.89)': {'lambda': 4.0},
        'Heavy (ρ≈0.98)': {'lambda': 4.4}
    }

    fig = make_subplots(rows=1, cols=3, shared_yaxes=True, subplot_titles=list(traffic_regimes.keys()))

    for i, (regime, regime_params) in enumerate(traffic_regimes.items()):
        base_params = {'mu1':4.0, 'mu2':5.0, 'p1':0.5, 'k':1, 'sim_time':STD_TIME, **regime_params}

        pre_means, non_means = [], []
        for st in st_range:
            raw_pre = run_replications_raw(run_manual_simulation, {**base_params, 'policy':'preemptive', 'switch_time':st}, STD_REPS)
            pre_means.append(np.mean([r['W_total'] for r in raw_pre]))
            raw_non = run_replications_raw(run_manual_simulation, {**base_params, 'policy':'non_preemptive', 'switch_time':st}, STD_REPS)
            non_means.append(np.mean([r['W_total'] for r in raw_non]))

        fig.add_trace(go.Scatter(x=st_range, y=pre_means, mode='lines+markers', name='Preemptive',
                                 line=dict(color=PROFESSIONAL_PALETTE['primary']), showlegend=(i==0)), row=1, col=i+1)
        fig.add_trace(go.Scatter(x=st_range, y=non_means, mode='lines+markers', name='Non-Preemptive',
                                 line=dict(color=PROFESSIONAL_PALETTE['secondary'], dash='dash'), showlegend=(i==0)), row=1, col=i+1)

    fig.update_layout(title_text="Policy Performance vs. Traffic Intensity", legend_title_text="Policy")
    fig.update_xaxes(title_text='Switch-over Time')
    fig.update_yaxes(title_text='Total Mean Sojourn Time E[W_total]', row=1, col=1)
    fig.show()

def run_sensitivity_heterogeneity():
    """
    Analyzes how the optimal K-threshold changes with service rate heterogeneity.
    """
    print("\n--- Running Sensitivity Analysis: System Heterogeneity ---")
    k_range_obj = range(1, 12)
    k_range_list = list(k_range_obj) # <-- FIX: Convert range object to a list

    heterogeneity_levels = {
        'Moderate (μ1=4.5, μ2=5.0)': {'mu1': 4.5, 'mu2': 5.0},
        'High (μ1=3.5, μ2=6.0)': {'mu1': 3.5, 'mu2': 6.0}
    }

    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=list(heterogeneity_levels.keys()))

    for i, (level, het_params) in enumerate(heterogeneity_levels.items()):
        base_params = {'lambda':4.0, 'p1':0.5, 'switch_time':0, 'policy':'non_preemptive', 'sim_time':STD_TIME, **het_params}

        w_totals = [np.mean([r['W_total'] for r in run_replications_raw(run_manual_simulation, {**base_params, 'k': k}, STD_REPS)]) for k in k_range_obj]
        optimal_k = k_range_list[np.argmin(w_totals)]

        # Use the converted list for plotting
        fig.add_trace(go.Scatter(x=k_range_list, y=w_totals, mode='lines+markers', name=level,
                                 line=dict(color=PROFESSIONAL_PALETTE['primary'])), row=1, col=i+1)
        fig.add_vline(x=optimal_k, line_width=2, line_dash="dash", line_color="red",
                      annotation_text=f"Optimal K* ≈ {optimal_k}", row=1, col=i+1)

    fig.update_layout(title_text="Optimal K vs. Service Rate Heterogeneity", showlegend=False)
    fig.update_xaxes(title_text='K Threshold')
    fig.update_yaxes(title_text='Total Mean Sojourn Time E[W_total]', row=1, col=1)
    fig.show()

In [ ]:
# =============================================================================
# CELL 9: Block 6 - Benchmark vs. M/G/1
# =============================================================================
def run_block_6_benchmark_vs_mg1():
    """
    Compares the JSQ-SLQ system against a standard M/G/1 queue benchmark.
    """
    print("\n--- Running Block 6: Benchmark vs. M/G/1 ---")
    p1_range = np.linspace(0.05, 0.55, 11)
    results_jsq, results_mg1 = [], []

    base_params = {'lambda': 4.0, 'mu1': 3.5, 'mu2': 5.0, 'sim_time': STD_TIME}
    params_jsq = {**base_params, 'k': 1, 'switch_time': 0, 'policy': 'preemptive'}

    for p1 in p1_range:
        raw_jsq = run_replications_raw(run_manual_simulation, {**params_jsq, 'p1': p1}, STD_REPS)
        for r in raw_jsq: r['L_total'] = r['L1'] + r['L2']
        results_jsq.append({'p1': p1, **calculate_stats(raw_jsq, 'L_total')})

        raw_mg1 = run_replications_raw(run_mg1_simulation, {**base_params, 'p1': p1}, STD_REPS)
        results_mg1.append({'p1': p1, **calculate_stats(raw_mg1, 'L_total')})

    df_jsq = pd.DataFrame(results_jsq)
    df_mg1 = pd.DataFrame(results_mg1)

    fig = go.Figure()
    def add_ci_trace(fig, df, name, color, fill, symbol, dash):
        fig.add_trace(go.Scatter(x=df['p1'], y=df['mean'] + df['ci_half_width'], mode='lines', line=dict(width=0), showlegend=False))
        fig.add_trace(go.Scatter(x=df['p1'], y=df['mean'], mode='lines+markers', name=name, line=dict(color=color, dash=dash),
                                  marker_symbol=symbol, fill='tonexty', fillcolor=fill))
        fig.add_trace(go.Scatter(x=df['p1'], y=df['mean'] - df['ci_half_width'], mode='lines', line=dict(width=0), fill='tonexty',
                                  fillcolor=fill, showlegend=False))

    add_ci_trace(fig, df_mg1, 'M/G/1 (Sim)', PROFESSIONAL_PALETTE['secondary'], PROFESSIONAL_PALETTE['error_fill_secondary'], 'square', 'dash')
    add_ci_trace(fig, df_jsq, 'JSQ-SLQ (Sim)', PROFESSIONAL_PALETTE['primary'], PROFESSIONAL_PALETTE['error_fill_primary'], 'circle', 'solid')

    fig.update_layout(title_text="Block 6: JSQ-SLQ vs. M/G/1 Benchmark", xaxis_title="Probability of Slower Server Task (p₁)",
                      yaxis_title="Total Mean Customers in System (E[L_total])", legend_title_text="Model")
    fig.show()

In [ ]:
# =============================================================================
# CELL 10: Block 7 - Animation of Queue Dynamics
# =============================================================================
def run_and_animate_queues():
    """
    Runs a simulation with logging and creates an animated bar chart of queue lengths.
    """
    print("\n--- Running Animation: Generating Queue Dynamics ---")
    anim_params = {
        'lambda': 4.0, 'mu1': 3.5, 'mu2': 6.0, 'p1': 0.5,
        'k': 3, 'switch_time': 0.1, 'policy': 'non_preemptive',
        'sim_time': 200, 'seed': 42,
        'log_data': True,
        'log_interval': 0.5
    }

    metrics, log_data = run_manual_simulation(anim_params)
    log_df = pd.DataFrame(log_data)

    if log_df.empty:
        print("Simulation log is empty. Cannot generate animation.")
        return

    anim_df = log_df.melt(id_vars=['time'], value_vars=['q1_len', 'q2_len'],
                          var_name='Queue', value_name='Length')
    anim_df['Queue'] = anim_df['Queue'].map({'q1_len': 'Queue 1 (Slow Server)', 'q2_len': 'Queue 2 (Fast Server)'})

    max_len = anim_df['Length'].max() if not anim_df.empty else 10
    y_range_max = max(10, np.ceil(max_len * 1.2))

    fig = px.bar(
        anim_df, x='Queue', y='Length', color='Queue', animation_frame='time',
        range_y=[0, y_range_max],
        color_discrete_map={
            'Queue 1 (Slow Server)': PROFESSIONAL_PALETTE['primary'],
            'Queue 2 (Fast Server)': PROFESSIONAL_PALETTE['secondary']
        },
        labels={'Length': 'Number of Customers in Queue'},
        title=f"Queue Dynamics Over Time (k={anim_params['k']})"
    )

    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 30
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 5
    fig.update_layout(xaxis_title="", showlegend=False)

    if fig.frames:
        for i, frame in enumerate(fig.frames):
             frame.layout.title.text = f"Queue Dynamics at Time: {frame.name}s (k={frame.name})"

    fig.show()

In [ ]:
# =============================================================================
# CELL 11: Main Execution Block
# =============================================================================
if __name__ == '__main__':
    # It is highly recommended to run these one at a time by uncommenting
    # the desired function call, as a full run will take several hours.

    print("--- Starting Final, High-Fidelity Simulation Suite ---")

    # --- Foundational Analyses ---
    run_block_1_validation()
    run_block_2_explore_k()

    # --- Core Research Questions ---
    run_block_5_policy_comparison()
    run_block_6_benchmark_vs_mg1()

    # --- Sensitivity and Optimization Analyses ---
    run_sensitivity_traffic_intensity()
    run_sensitivity_heterogeneity()
    run_pareto_frontier_analysis()

    # --- Visualization ---
    run_and_animate_queues()

    print("\n--- All selected experiments complete. ---")

--- Starting Final, High-Fidelity Simulation Suite ---

--- Running Block 1: Validation ---



--- Running Block 2: Impact of K-Threshold ---



--- Running Block 3 (Rigorous): Preemptive vs. Non-Preemptive ---
Running t-tests for each switch time point...
  Switch Time 0.00: Preemptive=2.324, Non-Preemptive=2.326. Significant Difference? No (p=0.8972)
  Switch Time 0.05: Preemptive=8.692, Non-Preemptive=8.940. Significant Difference? No (p=0.2580)
  Switch Time 0.10: Preemptive=1806.768, Non-Preemptive=1777.812. Significant Difference? No (p=0.2112)
  Switch Time 0.15: Preemptive=4067.814, Non-Preemptive=4079.076. Significant Difference? No (p=0.5920)
  Switch Time 0.20: Preemptive=5690.932, Non-Preemptive=5700.772. Significant Difference? No (p=0.6010)
  Switch Time 0.25: Preemptive=6833.922, Non-Preemptive=6836.732. Significant Difference? No (p=0.8842)



--- Running Block 6: Benchmark vs. M/G/1 ---



--- Running Sensitivity Analysis: Traffic Intensity (ρ) ---



--- Running Sensitivity Analysis: System Heterogeneity ---



--- Running Block 4: Pareto Frontier Analysis ---



--- Running Animation: Generating Queue Dynamics ---



--- All selected experiments complete. ---


In [6]:
# =============================================================================
# CELL 2 (REPLACEMENT): Definitive Simulation for Replicating Nir's Scenario
# =============================================================================

def run_nir_simulation_final_synthesis(params):
    """
    This is the definitive, synthesized simulation engine. It combines the validated logic from
    the original project code with the crucial corrections discovered during our analysis.
    1. K-threshold check correctly occurs upon SERVICE COMPLETION.
    2. JSQ arrival policy includes the CORRECT probabilistic tie-breaker.
    3. The "Bypassed Customers" metric, confirmed to be in Nir's model, is re-integrated.
    4. Metrics (L, Lq) are calculated with robust time-averaged integrals.
    5. A warmup period is implemented to ensure steady-state results.
    """
    # --- Event Constants ---
    ARRIVAL, DEPARTURE = 'arrival', 'departure'

    # --- Parameter Unpacking ---
    sim_time = params['sim_time']
    warmup_time = params.get('warmup_time', 0)

    # --- State Variables ---
    sim_random = random.Random(params['seed'])
    now, event_calendar, queues = 0.0, [], [deque(), deque()]
    server_location, server_status, current_customer = 0, 'IDLE', None

    # --- Metric Accumulators ---
    sojourn_times = [[], []]
    sys_len_integral, q1_len_integral, q2_len_integral = 0.0, 0.0, 0.0
    last_event_time = 0.0
    switch_count, bypassed_q1, bypassed_q2 = 0, 0, 0

    # --- Warmup State ---
    warmup_complete = False
    metrics_start_time = 0.0

    # --- HELPER FUNCTIONS ---
    def schedule_event(delay, event_type, data=None):
        heapq.heappush(event_calendar, (now + delay, event_type, data))

    def update_time_averaged_stats():
        nonlocal last_event_time, sys_len_integral, q1_len_integral, q2_len_integral
        if warmup_complete:
            time_delta = now - last_event_time
            q1_len, q2_len = len(queues[0]), len(queues[1])
            server_busy_flag = 1 if server_status == 'BUSY' else 0

            sys_len_integral += (q1_len + q2_len + server_busy_flag) * time_delta
            q1_len_integral += q1_len * time_delta
            q2_len_integral += q2_len * time_delta

        last_event_time = now

    def process_arrival(arrival_time):
        nonlocal server_status
        # JSQ policy with probabilistic tie-breaker
        len_q1, len_q2 = len(queues[0]), len(queues[1])
        if len_q1 < len_q2:
            chosen_q_idx = 0
        elif len_q2 < len_q1:
            chosen_q_idx = 1
        else: # Tie-breaker
            chosen_q_idx = 0 if sim_random.random() < params['p1'] else 1

        queues[chosen_q_idx].append(arrival_time)
        schedule_event(sim_random.expovariate(params['lambda']), ARRIVAL)

        if server_status == 'IDLE':
            start_service_if_possible()

    def start_service_if_possible():
        nonlocal server_status, current_customer, server_location, switch_count, bypassed_q1, bypassed_q2
        if server_status != 'IDLE': return

        # If server's current queue has customers, serve it.
        if len(queues[server_location]) > 0:
            # Re-integrate the bypass logic/metric from original code
            if warmup_complete:
                len_i, len_j = len(queues[server_location]), len(queues[1 - server_location])
                if len_j > len_i and (len_j - len_i < params['k']):
                    # Server is at Q_i, but Q_j is longer (but not by K).
                    # A customer from Q_i is served, "bypassing" the longer Q_j.
                    if server_location == 0: bypassed_q2 += 1
                    else: bypassed_q1 += 1

            arrival_time = queues[server_location].popleft()
            service_rate = params['mu1'] if server_location == 0 else params['mu2']
            server_status = 'BUSY'
            current_customer = {'arrival_time': arrival_time}
            schedule_event(sim_random.expovariate(service_rate), DEPARTURE, {'q_idx': server_location})

        # If current queue is empty, but the other has customers, switch and serve.
        elif len(queues[1 - server_location]) > 0:
            server_location = 1 - server_location
            if warmup_complete: switch_count += 1
            start_service_if_possible()

    def process_departure(data):
        nonlocal server_status, current_customer, server_location, switch_count

        if warmup_complete:
            sojourn_times[data['q_idx']].append(now - current_customer['arrival_time'])

        server_status, current_customer = 'IDLE', None

        # Corrected Logic: K-threshold check happens upon service completion.
        len_i, len_j = len(queues[server_location]), len(queues[1 - server_location])
        if (len_j - len_i >= params['k']):
            server_location = 1 - server_location
            if warmup_complete: switch_count += 1

        start_service_if_possible()

    # --- MAIN SIMULATION LOOP ---
    schedule_event(0, ARRIVAL)
    while event_calendar and now < sim_time:
        if not warmup_complete and now >= warmup_time:
            warmup_complete = True
            metrics_start_time = now
            last_event_time = now
            # Reset all metric accumulators
            sojourn_times = [[], []]
            switch_count, bypassed_q1, bypassed_q2 = 0, 0, 0
            sys_len_integral, q1_len_integral, q2_len_integral = 0.0, 0.0, 0.0

        now, event_type, data = heapq.heappop(event_calendar)
        if now >= sim_time: break

        update_time_averaged_stats()

        if event_type == ARRIVAL:
            process_arrival(now)
        elif event_type == DEPARTURE:
            process_departure(data)

    # --- FINAL METRIC CALCULATION ---
    metrics_duration = now - metrics_start_time
    if metrics_duration <= 0: return {}

    mean_Lq1 = q1_len_integral / metrics_duration
    mean_Lq2 = q2_len_integral / metrics_duration

    return {
        "Total Customers (Mean)": mean_Lq1 + mean_Lq2,
        "Queue 1 Length (Mean)": mean_Lq1,
        "Queue 2 Length (Mean)": mean_Lq2,
        "Sojourn Time Q1 (Mean)": np.mean(sojourn_times[0]) if sojourn_times[0] else 0,
        "Sojourn Time Q2 (Mean)": np.mean(sojourn_times[1]) if sojourn_times[1] else 0,
        "Switches per Time Unit (Mean)": switch_count / metrics_duration,
        "Bypassed Customers Q1 (Mean)": bypassed_q1,
        "Bypassed Customers Q2 (Mean)": bypassed_q2,
    }

# =============================================================================
# CELL 3 (REPLACEMENT): Analysis Function for Nir's Scenario
# =============================================================================

def run_nir_comparison_final_analysis_synthesis():
    """
    This function orchestrates the simulation run to replicate Nir's results.
    It uses the definitive simulation engine and the exact parameters provided.
    """
    print("\n--- Running The Definitive Final Comparison against Nir's Results ---")

    # Parameters as confirmed by Nir's files
    nir_params = {
        'lambda': 4.0,
        'mu1': 5.0,
        'mu2': 4.0,
        'p1': 0.5,
        'switch_time': 0, # Not used in the simplified model, but good practice
        'sim_time': 100000,
        'warmup_time': 10000,
    }

    k_range = range(3, 16)
    num_replications = 30
    all_results = []

    # Define the exact metrics to track, matching Nir's Excel file
    metrics_to_track = [
        "Total Customers (Mean)",
        "Queue 1 Length (Mean)",
        "Queue 2 Length (Mean)",
        "Sojourn Time Q1 (Mean)",
        "Sojourn Time Q2 (Mean)",
        "Switches per Time Unit (Mean)",
        "Bypassed Customers Q1 (Mean)",
        "Bypassed Customers Q2 (Mean)"
    ]

    for k in k_range:
        print(f"Running definitive synthesized simulations for K={k}...")
        raw_results = []
        for i in range(num_replications):
            # Pass a different seed for each replication
            raw_results.append(run_nir_simulation_final_synthesis({**nir_params, 'k': k, 'seed': i}))

        # Calculate mean and standard error for this K value
        k_summary = {'K': k}
        for metric in metrics_to_track:
            values = [res.get(metric, 0) for res in raw_results]
            k_summary[metric] = np.mean(values)
            k_summary[metric.replace("(Mean)", "(Std Error)")] = np.std(values, ddof=1) / np.sqrt(num_replications)

        all_results.append(k_summary)

    # Format and print the final results table
    results_df = pd.DataFrame(all_results).set_index('K')
    pd.set_option('display.float_format', '{:.7f}'.format)
    print("\n--- Definitive Final Synthesized Simulation Results ---")
    print(results_df)

# =============================================================================
# CELL 4 (REPLACEMENT): Main Execution Block
# =============================================================================

if __name__ == '__main__':
    # To replicate Nir's results, run this function
    run_nir_comparison_final_analysis_synthesis()

    # You can comment out the line above and uncomment the lines below
    # to run the original project experiments from your other cells if needed.

    # print("\n--- Starting Original Project Simulation Suite ---")
    # run_block_1_validation()
    # run_block_2_explore_k()
    # ... etc ...


--- Running The Definitive Final Comparison against Nir's Results ---
Running definitive synthesized simulations for K=3...
Running definitive synthesized simulations for K=4...
Running definitive synthesized simulations for K=5...
Running definitive synthesized simulations for K=6...
Running definitive synthesized simulations for K=7...
Running definitive synthesized simulations for K=8...
Running definitive synthesized simulations for K=9...
Running definitive synthesized simulations for K=10...
Running definitive synthesized simulations for K=11...
Running definitive synthesized simulations for K=12...
Running definitive synthesized simulations for K=13...
Running definitive synthesized simulations for K=14...
Running definitive synthesized simulations for K=15...

--- Definitive Final Synthesized Simulation Results ---
    Total Customers (Mean)  Total Customers (Std Error)  \
K                                                         
3                9.3121361                    